In [6]:
import pandas as pd

preferencias = pd.read_csv("data/usuarios_preferencias.csv", sep = ",", header = 0)
preferencias.columns = ['id_user', 'id_preferencia', 'score']
matriz_preferencias = preferencias.pivot(index='id_user', columns='id_preferencia', values='score')
columnas_deseadas = list(range(1, 116))
matriz_preferencias = matriz_preferencias.reindex(columns=columnas_deseadas, fill_value=0)
matriz_preferencias = matriz_preferencias.fillna(0)
matriz_preferencias = matriz_preferencias.T
matriz_preferencias

id_user,123,124,125,126,127,128,129,130,131,132,...,171,172,173,174,175,176,177,178,179,180
id_preferencia,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.0,0.0,0.0,0.0,9.0,8.0,4.0,0.0,8.0,9.0,...,8.0,8.0,9.0,8.0,0.0,5.0,8.0,10.0,0.0,7.0
112,0.0,0.0,0.0,3.0,5.0,5.0,7.0,0.0,9.0,4.0,...,6.0,5.0,8.0,8.0,0.0,8.0,0.0,5.0,0.0,8.0
113,0.0,0.0,0.0,5.0,5.0,4.0,5.0,0.0,3.0,10.0,...,3.0,6.0,7.0,9.0,0.0,10.0,7.0,5.0,0.0,6.0


In [7]:
padres = pd.read_csv("data/preferencias.csv")
padres.index = range(1,116)
#padres

In [8]:
matriz_preferencias_mayor0 = matriz_preferencias>0
matriz_preferencias_mayor0['padre'] = padres['parent']
matriz_preferencias_conteos = matriz_preferencias_mayor0.groupby('padre').sum()
#matriz_preferencias_conteos

In [ ]:
import pandas as pd

def filtrar_top_porcentaje(df, porcentaje=0.2):
    df_filtrado = df.copy()
    columnas_usuario = [col for col in df.columns if col != 'padre']
    
    # Agrupamos por la columna 'padre'
    for padre, grupo in df.groupby('padre'):
        indices_grupo = grupo.index  # Índices de este grupo
        
        for col in columnas_usuario:
            # Contamos cuántos valores ha puntuado cada usuario dentro del grupo (valores > 0)
            conteo_puntuaciones = (grupo[col] > 0).sum()
            
            # Calculamos cuántos elementos debemos conservar en base al porcentaje
            n_items = max(1, int(conteo_puntuaciones * porcentaje))
            
            if n_items > 0:
                # Seleccionamos los índices de los mayores valores puntuados por cada usuario
                indices_top = grupo[col].nlargest(n_items).index
                # El resto se pone a 0
                indices_no_top = grupo.index.difference(indices_top)
                df_filtrado.loc[indices_no_top, col] = 0
                
    return df_filtrado

matriz_preferencias['padre'] = padres['parent']

# Aplicamos la función con el porcentaje deseado (ejemplo 20%)
matriz_filtrada = filtrar_top_porcentaje(matriz_preferencias, porcentaje=0.2)

# Contamos cuántos elementos > 0 quedan en cada columna
#(matriz_filtrada > 0).sum()


In [19]:
import pandas as pd

items = pd.read_csv("data/clasificacion_items.csv", sep = ";", header = None)
items.columns = ['id_item', 'id_preferencia', 'score']
items = items.groupby(['id_item', 'id_preferencia'], as_index=False).mean()
matriz_items = items.pivot(index='id_item', columns='id_preferencia', values='score')
columnas_deseadas = list(range(1, 116))
matriz_items = matriz_items.reindex(columns=columnas_deseadas, fill_value=0)
matriz_items = matriz_items.fillna(0)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
matriz_filtradaT= matriz_filtrada.T
matriz_similitud_items = cosine_similarity(matriz_items.values, matriz_filtradaT.values) #coinciden en las columnas

matriz_similitud_items_df = pd.DataFrame(matriz_similitud_items, 
                                         index=matriz_items.index, 
                                         columns=matriz_filtradaT.index)

matriz_similitud_items_df = matriz_similitud_items_df.T 
#matriz_similitud_items_df

In [54]:
items_names = pd.read_csv("data/items.csv", header = 0, sep = ",")
items_names.columns = ['id_item', 'name_item', 'visitas']
items_names

,id_item,name_item,visitas
0,17365,Casa de las Rocas-Museo del Corpus,11
1,17366,Casa-Museo Benlliure,11
2,17367,Casa-Museo Blasco Ibañez,11
3,17369,Centro cultural Fundación Bancaja,30
4,17371,Centro del Carmen-IVAM,26
...,...,...,...
144,18001,Barrio del Carmen,0
145,18002,Barrio de Ruzafa,0
146,18003,La Albufera,0
147,18004,Barrio del Cabañal,0


In [55]:
usuarios_historico = pd.read_csv("data/puntuaciones_usuario_base.csv", sep = ",", header = 0)
usuarios_historico.columns = ['id_user', 'id_item', 'valoracion', "valoracion_norm"]
#usuarios_historico

In [66]:
import numpy as np
import pandas as pd

# Cargar el histórico de usuarios desde el CSV
usuarios_historico = pd.read_csv("data/puntuaciones_usuario_base.csv", sep = ",", header = 0)
usuarios_historico.columns = ['id_user', 'id_item', 'valoracion', "valoracion_norm"]
usuarios_historico = usuarios_historico.drop("valoracion_norm", axis=1)

def obtener_historial(usuarios_historico):
    """
    Convierte el DataFrame de histórico en un diccionario {usuario_id: lista de ítems visitados}
    """
    return usuarios_historico.groupby('id_user')['id_item'].apply(list).to_dict()

def recomendar_items(user_id, matriz_similitud, usuarios_historico, items_names, N=5):
    """
    Recomienda los N ítems más similares a los que el usuario ha visitado, evitando recomendar ítems ya vistos.

    Parámetros:
    - user_id: ID del usuario para el que se hace la recomendación.
    - matriz_similitud: DataFrame de similitud entre las preferencias de los usuarios y los items.
    - usuarios_historico: DataFrame con historial de ítems visitados por usuarios.
    - items_names: DataFrame con los nombres de los ítems.
    - N: Número de recomendaciones a mostrar.

    Retorna:
    - Lista de tuplas con (nombre_item, ratio_similitud)
    """
    
    # Obtener el historial de cada usuario como un diccionario
    historial_usuarios = obtener_historial(usuarios_historico)
    
    # Obtener los ítems ya visitados por el usuario
    items_visitados = historial_usuarios.get(user_id, [])
    
    # Definir colores ANSI
    COLOR_TITULO = "\033[1;32m"    # Verde brillante
    COLOR_ITEM   = "\033[1;34m"    # Azul brillante
    COLOR_VAL    = "\033[1;33m"    # Amarillo brillante
    COLOR_RESET  = "\033[0m"       # Reset

    # 3. Mostrar los gustos o preferencias del usuario obtenidos de la matriz_filtrada
    # Suponemos que 'matriz_filtrada' tiene índice = id_preferencia y columnas = id_user
    if user_id in matriz_filtrada.columns:
        gustos_usuario = matriz_filtrada[user_id]
        # Si existe una columna 'padre' en el índice, la descartamos:
        if 'padre' in gustos_usuario.index:
            gustos_usuario = gustos_usuario.drop('padre')
        # Seleccionar preferencias con valor > 0
        gustos_usuario = gustos_usuario[gustos_usuario > 0]
        if not gustos_usuario.empty:
            # Convertir a DataFrame y fusionar con 'padres' para obtener nombres
            gustos_usuario_df = pd.DataFrame(gustos_usuario)
            gustos_usuario_df = gustos_usuario_df.rename(columns={user_id: 'score'})
            gustos_usuario_df.index.name = 'id_preferencia'
            # Aseguramos que en 'padres' la columna que identifica la preferencia se llame "id" y la ponemos como índice
            padres_indexed = padres.set_index('id')
            gustos_usuario_df = gustos_usuario_df.merge(padres_indexed, left_index=True, right_index=True, how='left')
            
            print(f"\n{COLOR_TITULO}Gustos o preferencias del usuario:{COLOR_RESET}")
            for idx, row in gustos_usuario_df.iterrows():
                print(f"{COLOR_ITEM}{row['name']}{COLOR_RESET} (id_preferencia: {idx}) - Score: {COLOR_VAL}{row['score']}{COLOR_RESET}")
    
    print("\n")
    print(f"{COLOR_TITULO}Items visitados por el usuario:{COLOR_RESET}")
    for item in items_visitados:
        # Obtener el nombre del ítem y la valoración
        nombre_item = items_names.loc[items_names['id_item'] == item]['name_item'].values[0]
        valoracion = usuarios_historico.loc[
            (usuarios_historico['id_user'] == user_id) & (usuarios_historico['id_item'] == item)
        ]['valoracion'].values[0]

        # Imprimir con colores
        print(f"{COLOR_ITEM}{nombre_item}{COLOR_RESET} - {COLOR_VAL}Valoración: {valoracion}{COLOR_RESET}")


    # Verificar que el usuario haya visitado algún ítem antes
    if not items_visitados:
        print("El usuario no tiene historial de visitas.")
        return []
    
    # Obtener la similitud de los ítems NO visitados por el usuario
    similar_items = matriz_similitud.loc[user_id].drop(index=items_visitados, errors='ignore')

    # Seleccionar los N ítems más similares
    top_items = similar_items.nlargest(N)
    print(top_items.to_dict())

    # Convertir los IDs en nombres de ítems
    resultado = [(items_names.loc[items_names['id_item'] == item, 'name_item'].values[0], ratio)
                 for item, ratio in top_items.items() if item in items_names['id_item'].values]

    return resultado  

# Ejemplo de uso:
user_id = 142  # ID del usuario
N = 10  # Número de recomendaciones

recomendaciones = recomendar_items(user_id, matriz_similitud_items_df, usuarios_historico, items_names, N)


COLOR_TITULO     = "\033[1;32m"  
COLOR_ITEM       = "\033[1;34m"  
COLOR_SIMILITUD  = "\033[1;33m"  
COLOR_RESET      = "\033[0m"     

print("\n")
for nombre_item, ratio in recomendaciones:
    print(f"{COLOR_TITULO}Recomendado:{COLOR_RESET} {COLOR_ITEM}{nombre_item}{COLOR_RESET} {COLOR_SIMILITUD}(Similitud: {ratio:.4f}){COLOR_RESET}")


Gustos o preferencias del usuario:
Prehistórico (id_preferencia: 16) - Score: 8.0
Árabe (id_preferencia: 17) - Score: 8.0
Parques (id_preferencia: 28) - Score: 8.0
Jardines botánicos (id_preferencia: 29) - Score: 10.0
Teatros (id_preferencia: 38) - Score: 10.0
Restaurantes (id_preferencia: 45) - Score: 6.0
Tiendas especializadas (id_preferencia: 47) - Score: 5.0
Exposiciones (id_preferencia: 50) - Score: 7.0
Nauticos (id_preferencia: 57) - Score: 3.0
Ciencia y tecnología (id_preferencia: 63) - Score: 10.0
Ciencias naturales (id_preferencia: 66) - Score: 10.0
Catedrales (id_preferencia: 72) - Score: 6.0
Iglesias (id_preferencia: 73) - Score: 5.0
Monasterios (id_preferencia: 75) - Score: 5.0
Castillos (id_preferencia: 87) - Score: 7.0
Palacio (id_preferencia: 91) - Score: 7.0
Casa (id_preferencia: 92) - Score: 7.0
Esculturas (id_preferencia: 105) - Score: 7.0
Centros de salud y spa (id_preferencia: 108) - Score: 1.0


Items visitados por el usuario:
Casa-Museo Benlliure - Valoración: 6


In [62]:
d = {}
for nombre_item, ratio in recomendaciones:
    d[nombre_item] = round(ratio, 4)
d

{'Jardín Botánico': 0.3172,
 'Teatro Olympia': 0.3038,
 'Museo Valenciano de Historia Natural': 0.3037,
 'Teatro Principal': 0.2942,
 'Teatro Rialto': 0.2837,
 'Jardines de la Glorieta': 0.2537,
 'Baños del Almirante': 0.2537,
 'El Almudín': 0.2501,
 'Iglesia y Palacio del Temple': 0.2282,
 'Teatro de Marionetas La Estrella': 0.2243}

In [64]:
recomendaciones

[('Jardín Botánico', 0.3171807398477756),
 ('Teatro Olympia', 0.30380404979209136),
 ('Museo Valenciano de Historia Natural', 0.3036893111306844),
 ('Teatro Principal', 0.29419112912796413),
 ('Teatro Rialto', 0.283695078181321),
 ('Jardines de la Glorieta', 0.25374459187822046),
 ('Baños del Almirante', 0.25374459187822046),
 ('El Almudín', 0.25012003270712824),
 ('Iglesia y Palacio del Temple', 0.22816432199007003),
 ('Teatro de Marionetas La Estrella', 0.2242806520081283)]

In [57]:
matriz_similitud_items_df = matriz_similitud_items_df.drop('padre')

In [58]:
# Fusionamos el histórico con las visitas para ponderar cada rating
interacciones = pd.merge(usuarios_historico, items_names[['id_item', 'visitas']], on='id_item', how='left')
interacciones['visitas'] = interacciones['visitas'].fillna(0)
# Fórmula de ponderación: rating * log(1+visitas)
interacciones['weighted_rating'] = interacciones.apply(
    lambda row: row['valoracion'] * np.log(1 + row['visitas'])
    if row['valoracion'] >= 4
    else -1 * row['valoracion'] * np.log(1 + row['visitas']),
    axis=1
)
interacciones

,id_user,id_item,valoracion,visitas,weighted_rating
0,123,17376,6,26,19.775021
1,123,17377,6,19,17.974394
2,123,17380,6,28,20.203775
3,123,17392,3,13,-7.917172
4,123,17395,4,19,11.982929
...,...,...,...,...,...
1109,180,17534,1,3,-1.386294
1110,180,17544,1,27,-3.332205
1111,180,17546,1,16,-2.833213
1112,180,17551,6,33,21.158163


In [12]:
matriz_interacciones = interacciones.pivot(index='id_user', columns='id_item', values='weighted_rating')
matriz_interacciones = matriz_interacciones.fillna(0)

In [13]:
import pandas as pd
import numpy as np

# 1. Cargar el DataFrame de ítems (clasificación) con las columnas: id_item, id_padre, score
items = pd.read_csv("clasificacion_items.csv", sep=";", header=None)
items.columns = ['id_item', 'id_padre', 'score']

# 2. Unir interacciones con items para obtener la categoría 'id_padre' y el score de cada ítem
interacciones_con_padre = pd.merge(interacciones, items[['id_item', 'id_padre', 'score']], 
                                   on='id_item', how='left')

# 3. Para cada id_item (que puede tener varios registros por distintos id_padre),
# calcular la suma total de "score" para distribuir el weighted_rating.
interacciones_con_padre['total_score'] = interacciones_con_padre.groupby('id_item')['score'].transform('sum')

# 4. Crear una nueva columna que distribuya el weighted_rating de forma proporcional:
interacciones_con_padre['weighted_rating_final'] = interacciones_con_padre['weighted_rating'] * \
    (interacciones_con_padre['score'] / interacciones_con_padre['total_score'])

# 5. Agrupar por usuario y por 'id_padre', sumando los weighted_rating_final
agrupado = interacciones_con_padre.groupby(['id_user', 'id_padre'])['weighted_rating_final'].sum().reset_index()

# 6. Pivotar para crear el vector de cada usuario: filas = usuarios, columnas = id_padre
vector_por_usuario = agrupado.pivot(index='id_user', columns='id_padre', values='weighted_rating_final').fillna(0)

# 7. Asegurarse de que las columnas sean del 1 al 115 (incluso si alguna categoría no aparece, se completa con 0)
columnas_deseadas = list(range(1, 116))
vector_por_usuario = vector_por_usuario.reindex(columns=columnas_deseadas, fill_value=0)

print("Vector de cada usuario (agrupado por id_padre, columnas del 1 al 115):")
vector_por_usuario.T.describe(percentiles = [0.9])

Vector de cada usuario (agrupado por id_padre, columnas del 1 al 115):


id_user,123,124,125,126,127,128,129,130,131,132,...,171,172,173,174,175,176,177,178,179,180
count,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,...,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000
mean,0.146374,0.235029,0.127607,0.136879,0.253199,0.248691,0.184998,0.113296,-0.029077,-0.006175,...,0.045232,0.048536,0.029848,0.184463,0.114762,0.165556,-0.053155,-0.052723,-0.045877,0.149101
std,0.441830,0.643790,0.500025,0.566093,0.643227,0.649152,0.575464,0.460899,0.146944,0.228908,...,0.325064,0.320422,0.446914,0.645083,0.640393,0.522474,0.128643,0.117666,0.163550,0.520094
min,-0.556800,-0.199174,-0.646899,-0.799298,-0.507523,-0.624116,-0.635503,-0.800687,-0.930885,-0.437110,...,-0.485885,-0.518178,-1.031021,-0.516018,-1.033623,-0.717000,-0.680535,-0.600806,-1.009414,-0.555855
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90%,0.643720,0.822318,0.682241,0.478631,1.231777,1.165181,0.931929,0.496234,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.693747,0.442734,0.873767,0.000000,0.000000,0.000000,0.655663
max,1.953363,4.669623,3.943983,4.669623,3.096241,3.624455,4.195809,2.276355,0.418853,1.834221,...,2.442374,2.035312,2.734268,4.882504,5.725655,2.599302,0.000000,0.000000,0.591925,2.708188


In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


matriz_filtradaT = matriz_filtrada.T
# Calculamos la similitud entre ítems basada en los perfiles de los ítems y la matriz filtrada
matriz_similitud_items = cosine_similarity(matriz_items.values, matriz_filtradaT.values)
matriz_similitud_items_df = pd.DataFrame(matriz_similitud_items, 
                                         index=matriz_items.index, 
                                         columns=matriz_filtradaT.index)
# Trasponemos para tener: filas = usuarios, columnas = ítems
matriz_similitud_items_df = matriz_similitud_items_df.T

# Verificamos las dimensiones de ambas matrices
print("Forma de la matriz de interacciones:", matriz_interacciones.shape)
print("Forma de la matriz de similitud:", matriz_similitud_items_df.shape)

# --- Alineación: Asegurarse de que ambas matrices tengan los mismos ítems (columnas)
# Si las columnas (id_item) no coinciden, se reindexa:
items_comunes = matriz_interacciones.columns.intersection(matriz_similitud_items_df.columns)
matriz_interacciones_alineada = matriz_interacciones[items_comunes]
matriz_similitud_items_df_alineada = matriz_similitud_items_df[items_comunes]

# --- Normalización 
def normalize_df(df):
    # Se normaliza cada DataFrame a rango [0, 1]
    min_val = df.values.min()
    max_val = df.values.max()
    return (df - min_val) / (max_val - min_val)

matriz_interacciones_norm = normalize_df(matriz_interacciones_alineada)
matriz_similitud_norm = normalize_df(matriz_similitud_items_df_alineada)

# --- Combinar ambas matrices con un parámetro alpha
alpha = 0.5  # Puedes ajustar este parámetro según tus experimentos
final_matrix = alpha * matriz_interacciones_norm + (1 - alpha) * matriz_similitud_norm

#final_matrix

Forma de la matriz de interacciones: (58, 139)
Forma de la matriz de similitud: (59, 150)


In [15]:
# Ejemplo de uso:
user_id = 142  # ID del usuario
N = 10  # Número de recomendaciones

recomendaciones = recomendar_items(user_id, final_matrix, usuarios_historico, items_names, N)

COLOR_TITULO     = "\033[1;32m"  
COLOR_ITEM       = "\033[1;34m"  
COLOR_SIMILITUD  = "\033[1;33m"  
COLOR_RESET      = "\033[0m"     

print("\n")
for nombre_item, ratio in recomendaciones:
    print(f"{COLOR_TITULO}Recomendado:{COLOR_RESET} {COLOR_ITEM}{nombre_item}{COLOR_RESET} {COLOR_SIMILITUD}(Similitud: {ratio:.4f}){COLOR_RESET}")


Gustos o preferencias del usuario:
Prehistórico (id_preferencia: 16) - Score: 8.0
Árabe (id_preferencia: 17) - Score: 8.0
Parques (id_preferencia: 28) - Score: 8.0
Jardines botánicos (id_preferencia: 29) - Score: 10.0
Teatros (id_preferencia: 38) - Score: 10.0
Restaurantes (id_preferencia: 45) - Score: 6.0
Tiendas especializadas (id_preferencia: 47) - Score: 5.0
Exposiciones (id_preferencia: 50) - Score: 7.0
Nauticos (id_preferencia: 57) - Score: 3.0
Ciencia y tecnología (id_preferencia: 63) - Score: 10.0
Ciencias naturales (id_preferencia: 66) - Score: 10.0
Catedrales (id_preferencia: 72) - Score: 6.0
Iglesias (id_preferencia: 73) - Score: 5.0
Monasterios (id_preferencia: 75) - Score: 5.0
Castillos (id_preferencia: 87) - Score: 7.0
Palacio (id_preferencia: 91) - Score: 7.0
Casa (id_preferencia: 92) - Score: 7.0
Esculturas (id_preferencia: 105) - Score: 7.0
Centros de salud y spa (id_preferencia: 108) - Score: 1.0


Items visitados por el usuario:
Casa-Museo Benlliure - Valoración: 6
